In [ ]:
input_file = 'results-p8.json'; #put input file path here

# Loading info from the input file 

In [ ]:
import json

base_model = [] 
batch_size = []
n_gpus = []
rate = []
repeats = []
stage = []

data = json.load(open(input_file))

for elem in data:
    base_model.append(elem['base_model'])
    batch_size.append(elem['batch_size'])
    n_gpus.append(elem['n_gpus'])
    rate.append(elem['rate'])
    repeats.append(elem['repeats'])
    stage.append(elem['stage'])

base_model_set = set(base_model)
base_model_set = sorted(base_model_set)

model_group = [] #group the data according to models
for elem in base_model_set:
    temp = [i for i in data if i['base_model'] == elem]
    model_group.append(temp)


batch_set = set(batch_size)
batch_set = sorted(batch_set)

n_gpus_set = set(n_gpus)
n_gpus_set = sorted(n_gpus_set)

stage_set = set(stage)

repeat_set = set(repeats)

In [ ]:
print("===Base model options===")
print(base_model_set)

print("\n===Batch size options===")
print(batch_set)

print("\n===Gpu number options===")
print(n_gpus_set)

print("\n===Stage options===")
print(stage_set)

print("\n===Repeat options===")
print(repeat_set)

In [ ]:
def extract_meta(model_group):
    for elem in model_group:
        print("=== Model ", elem[0]['base_model'], " ===")
        for sing in elem:
            for key in sing.keys():
                if (key != 'times'):
                    print(key, sing[key], end='  ')
            print("\n")
                    
        print("\n\n")
extract_meta(model_group)

# Generate diagrams
Will show up on a new tab.

In [ ]:
from bokeh.io import show, output_file
from bokeh.models import FactorRange
from bokeh.plotting import figure
from bokeh.layouts import column, gridplot

output_file_name = (input_file.split('.'))[0]
output_file(output_file_name +".html")
def plot_model_times(model_data):
    factors1 = [
        ("FW", "1"),("FW", "2"), ("FW", "3"),("FW", "4"),
        ("TRAIN", "1"),("TRAIN", "2"), ("TRAIN", "3"),("TRAIN", "4"),
    ]
    times_min = []
    times_avg = []
    for elem in model_data:
        if(elem['stage'] == 'fw'):
            times_min.append(min(elem['times']))
            times_avg.append(sum(elem['times'])/100.0)
    for elem in model_data:
        if(elem['stage'] == 'train'):
            times_min.append(min(elem['times']))
            times_avg.append(sum(elem['times'])/100.0)

    
# output to static HTML file
    #output_file("line.html")
    name = model_data[0]['base_model']+ " batch_size = "+str(model_data[0]['batch_size'])

    p = figure(title=name,x_range=FactorRange(*factors1),x_axis_label='n_gpus', plot_height=400)
    p.vbar(x=factors1, top=times_avg, width=0.9, alpha=0.5, color = 'blue',legend="times_avg")
    p.vbar(x=factors1, top=times_min, width=0.9, alpha=0.5, color = 'pink',legend="times_min")
    p.legend.location = "top_left"
    p.x_range.range_padding = 0.1
    
    
    factors2= [
        ("FW_RATE","1"),("FW_RATE", "2"), ("FW_RATE", "3"),("FW_RATE", "4"),
        ("TRAIN_RATE", "1"),("TRAIN_RATE", "2"), ("TRAIN_RATE", "3"),("TRAIN_RATE", "4")
    ]
    
    factors_fw_rate = [("FW_RATE","1"),("FW_RATE", "2"), ("FW_RATE", "3"),("FW_RATE", "4")]
    factors_train_rate = [("TRAIN_RATE", "1"),("TRAIN_RATE", "2"), ("TRAIN_RATE", "3"),("TRAIN_RATE", "4")]
    rate = []
    rate_fw = []
    rate_train = []
    for elem in model_data:
        if(elem['stage'] == 'fw'):
            rate.append(elem['rate'])
            rate_fw.append(elem['rate'])
    for elem in model_data:
        if(elem['stage'] == 'train'):
            rate.append(elem['rate'])
            rate_train.append(elem['rate'])


    
# output to static HTML file
    #output_file("line.html")
    machine = 8
    if(input_file.find('9')>0):
        machine = 9
    name = "===P"+str(machine)+"=== "+model_data[0]['base_model']+ " batch_size = "+str(model_data[0]['batch_size'])

    p2 = figure(title=name,x_range=FactorRange(*factors2),x_axis_label='n_gpus', plot_height=400)
    p2.vbar(x=factors2, top=rate, width=0.9, alpha=0.5, color = 'blue')
    p2.line(x=factors_fw_rate, y=rate_fw, line_width=0.9, color = 'red')
    p2.line(x=factors_train_rate, y=rate_train, line_width=0.9, color = 'red')
    p2.x_range.range_padding = 0.1
# show the results

    
    return [p,p2]

figures = []
for elem in model_group:
    figures.append(plot_model_times(elem))
    
p = gridplot(figures)
show(p)